In [ ]:
import os
import openai

os.environ["CUDA_VISIBLE_DEVICES"] = "6"
os.environ["OPENAI_KEY"] = "xxxx" # xxxx is your api
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "omniqa-xxxxxx.json" # a json file looks like omniqa-example.json
openai.api_key = os.environ.get('OPENAI_KEY')

import cv2
import json
import numpy as np
from os.path import join as pjoin
import glob
import sys
import time
import datetime
from PIL import Image
import requests
from flask.views import MethodView

from flask import Flask, request, jsonify
import json
from assistgui.planner.assistsoftware_step import AssistGUI
from threading import Thread


def get_current_time():
    return datetime.datetime.now().strftime('%Y%m%d_%H%M%S')


class Server(MethodView):
    def __init__(self, target_window='Adobe Premiere Pro'):
        # judge if the cache folder exists
        global cache_folder
        self.cache_folder = cache_folder
        self.trust_threshold = 0
        if not os.path.exists(self.cache_folder):
            os.makedirs(self.cache_folder)
    
    def post(self):
        meta_data, query, task_id, reset, software, screenshot = self.recieve_data(request)
        if reset:
            print("reset planner")
            react.reset()
            
        meta_data_path, screenshot_path = self.save_data(meta_data, screenshot, react)
        
        react.software_name = software

        use_gt_plan = False  # set to True if use gt plan
        use_gt_action = False  # set to True if use gt action
        gt_path = None  # provide if needed
        ace_path = None  # provide if needed
        gtwgui_path = None  # provide if needed
        
        if_check = False
        
        if react.step < 1:
            if task_id is not None:
                instructional_video_path = glob.glob(f"../client/TrustBench/projects/{task_id}.*/{task_data[task_id]['video']}")[0]
            else:
                instructional_video_path = None
            response = react.run_step(query, meta_data, screenshot_path, instructional_video_path=instructional_video_path, task_id=task_id, use_gt_plan=use_gt_plan, use_gt_action=use_gt_action, gt_path=gt_path, ace_path=ace_path, gtwgui_path=gtwgui_path, if_check=if_check)
            response_returned = response
        else:
            # Trustworthy AI
            response = react.run_step(query, meta_data, screenshot_path, task_id=task_id, use_gt_plan=use_gt_plan, use_gt_action=use_gt_action, gt_path=gt_path, ace_path=ace_path, gtwgui_path=gtwgui_path, if_check=if_check)
            trust_score = float(response['trust_score'])
            
            if trust_score > self.trust_threshold:
                # return AI action
                response_returned = response
            else:
                # return Human action
                response_returned = [""]
                
        return jsonify(response_returned), 200

    def recieve_data(self, request):
        print("receiving data ... ")        
        meta_data = json.loads(request.form['data'])
        if 'iSlide' in meta_data:
            del meta_data['iSlide']
        with open(os.path.join(self.cache_folder, f"meta-data-{react.step}.json"), "w") as f:
            json.dump(meta_data, f)

        query = json.loads(request.form['query'])
        task_id = json.loads(request.form['task_id'])
        reset = json.loads(request.form['is_plan'])
        software = json.loads(request.form['software'])
        screenshot = request.files['image']

        print("Query: ", query)
        print("Task ID: ", task_id)
        print("Reset: ", reset)
        print("Software: ", software)

        return meta_data, query, task_id, reset, software, screenshot

    def save_data(self, meta_data, screenshot, react):
        meta_data_path = os.path.join(self.cache_folder, f"meta-data-{react.step}.json")
        json.dump(meta_data, open(meta_data_path, "w"))

        screenshot_path = os.path.join(self.cache_folder, f"screenshot-{react.step}.png")
        screenshot.save(screenshot_path)

        print(f"Save data successfully to {self.cache_folder}")
        return meta_data_path, screenshot_path


app = Flask(__name__)

cache_folder = os.path.join(".cache/", get_current_time())

upload_view = Server.as_view('upload_api')
app.add_url_rule('/api/upload', view_func=upload_view, methods=['POST'])

def run_flask():
    app.run(host='0.0.0.0', port=4338)
    
react = AssistGUI()

task_data = json.load(open("../client/TrustBench/ace_test.json"))

thread = Thread(target=run_flask)
thread.start()